<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/You_Tube_video_transcript_Summarizer_using_whisper_and_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install -qU openai langchain tiktoken pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00


In [7]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.1 MB/s eta 0:00:00


In [1]:
! pip install youtube_transcript_api

In [8]:
import os
import openai
from getpass import getpass
#
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
#
import re
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
import textwrap
from youtube_transcript_api import YouTubeTranscriptApi
#
from pytube import YouTube
import whisper

## Set up Openai Key

In [21]:
os.environ["OPENAI_API_KEY"] = getpass()

··········


## Instantiate models

In [9]:
def load_models():
  model = whisper.load_model("large")
  return model

In [10]:
model = load_models()

100%|██████████████████████████████████████| 2.87G/2.87G [00:29<00:00, 105MiB/s]


In [11]:
import tiktoken
tokenizer = tiktoken.get_encoding('cl100k_base')
#
def tiktoken_len(text):
  tokens = tokenizer.encode(text,disallowed_special=())
  return len(tokens)

In [18]:
def get_youtube_transcript(video_url):

    pattern = r"(?<=v=)[\w-]+|(?<=youtu.be/)[\w-]+"
    match = re.search(pattern, url)
    video_id = match.group()

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        context = []
        for line in transcript:
            context.append(line['text'])
        content = " ".join(context)
        return content
    except Exception as e:
        print(f"Error fetching transcript: {e}")
        return None

In [68]:
from pytube import YouTube

def get_youtube_metadata(video_url):
    try:
        yt = YouTube(video_url)
        yt.check_availability()
        metadata = {
            'title': yt.title,
            'description': yt.description,
            'author': yt.author,
        }
        return metadata
    except Exception as e:
        print(f"Error fetching metadata: {e}")
        return None


In [69]:
url = "https://youtu.be/I2ZK3ngNvvI"
get_youtube_metadata(url)

{'title': 'Advice for machine learning beginners | Andrej Karpathy and Lex Fridman',
 'description': None,
 'author': 'Lex Clips'}

In [70]:
def summarize_yt(url,model):
  yt = YouTube(url)
  audio = yt.streams.filter(only_audio=True).first().download()
  print("Audio downloaded")

  audio_transcript = model.transcribe(audio,fp16=False)['text']
  print(audio_transcript)

  text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,
                                                 chunk_overlap=20,
                                                 length_function = tiktoken_len,
                                                 separators = ['\n\n','\n','  ',' '])
  texts = text_splitter.split_text(audio_transcript)
  docs = [Document(page_content=t) for t in texts]
  #
  transcript = get_youtube_transcript(url)
  #
  metadata = get_youtube_metadata(url)
  print("Text split")
  return (docs,transcript,metadata)

In [74]:
def summarize(data):
  #
  docs,transcript,metadata = data
  prompt_template = """You are an expert at making strong factual summarizations.Please provide a meaningful and concise summary base on the VIDEO TRANSCRIPT enclosed within 3 single quotes, the VIDEO Metadata and
  text provided enclosed within 3 bacticks. Along with the summary please provide a point by point 10 key takeways from the video transcript and text. Don't include opinions.
  Then Give each key takeaways a number and keep them direct and to the point
  VIDEO Metadata : The tile of the video is {metadata}
  VIDEO TRANSCRIPT: '''{transcript}'''
  TEXT :```{text}```"""
  prompt_template = prompt_template.replace("{transcript}",transcript).replace("{metadata}",metadata['title'])
  PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
  print(PROMPT)
  #
  llm = ChatOpenAI(model="gpt-4", temperature=0.0)
  #
  chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)
  #
  response = chain.run(docs)
  return response



## Transcribe the Youtube link provided

In [71]:
%%time
url = "https://youtu.be/I2ZK3ngNvvI"
docs = summarize_yt(url,model)

Audio downloaded
 You're one of the greatest teachers of machine learning, AI ever, from CS231N to today. What advice would you give to beginners interested in getting into machine learning? Beginners are often focused on like what to do, and I think the focus should be more like how much you do. So I am kind of like believer on a high level in this 10,000 hours kind of concept where just kind of have to just pick the things where you can spend time and you care about and you're interested in. You literally have to put in 10,000 hours of work. It doesn't even matter as much like where you put it and you'll iterate and you'll improve and you'll waste some time. I don't know if there's a better way. You need to put in 10,000 hours, but I think it's actually really nice because I feel like there's some sense of determinism about being an expert at a thing. If you spend 10,000 hours, you can literally pick an arbitrary thing. And I think if you spend 10,000 hours of deliberate effort and w

## Response generated by Davinci

In [37]:
%%time
response = summarize(docs)
print(response)

input_variables=['text'] output_parser=None partial_variables={} template="You are an expert at making strong factual summarizations.Please provide a meaningful and concise summary base on the VIDEO TRANSCRIPT enclosed within 3 single quotes and \n  text provided enclosed within 3 bacticks. Along with the summary please provide a point by point 10 key takeways from the video transcript and text. Don't include opinions.\n  Give each key takeaways a number and keep them short and to the point.\n\n  VIDEO TRANSCRIPT: '''you're one of the greatest teachers of machine learning AI ever from cs231n to today what advice would you give to beginners interested in getting into machine learning beginners are often focused on like what to do and I think the focus should be more like how much you do so I I'm kind of like believer on a high level in this 10 000 hours kind of concept where you just kind of have to just pick the things where you can spend time and you you care about and you're interest

## Response Generated by GPT3

In [38]:
%%time
response = summarize(docs)
print(response)

input_variables=['text'] output_parser=None partial_variables={} template="You are an expert at making strong factual summarizations.Please provide a meaningful and concise summary base on the VIDEO TRANSCRIPT enclosed within 3 single quotes and \n  text provided enclosed within 3 bacticks. Along with the summary please provide a point by point 10 key takeways from the video transcript and text. Don't include opinions.\n  Give each key takeaways a number and keep them short and to the point.\n\n  VIDEO TRANSCRIPT: '''you're one of the greatest teachers of machine learning AI ever from cs231n to today what advice would you give to beginners interested in getting into machine learning beginners are often focused on like what to do and I think the focus should be more like how much you do so I I'm kind of like believer on a high level in this 10 000 hours kind of concept where you just kind of have to just pick the things where you can spend time and you you care about and you're interest

## Response generated by GP4

In [75]:
%%time
response = summarize(docs)
print(response)

input_variables=['text'] output_parser=None partial_variables={} template="You are an expert at making strong factual summarizations.Please provide a meaningful and concise summary base on the VIDEO TRANSCRIPT enclosed within 3 single quotes, the VIDEO Metadata and \n  text provided enclosed within 3 bacticks. Along with the summary please provide a point by point 10 key takeways from the video transcript and text. Don't include opinions.\n  Then Give each key takeaways a number and keep them direct and to the point\n  VIDEO Metadata : The tile of the video is Advice for machine learning beginners | Andrej Karpathy and Lex Fridman \n  VIDEO TRANSCRIPT: '''you're one of the greatest teachers of machine learning AI ever from cs231n to today what advice would you give to beginners interested in getting into machine learning beginners are often focused on like what to do and I think the focus should be more like how much you do so I I'm kind of like believer on a high level in this 10 000 

In [44]:
%%time
response = summarize(docs)
print(response)

input_variables=['text'] output_parser=None partial_variables={} template="You are an expert at making strong factual summarizations.Please provide a meaningful and concise summary base on the VIDEO TRANSCRIPT enclosed within 3 single quotes and \n  text provided enclosed within 3 bacticks. Along with the summary please provide a point by point 10 key takeways from the video transcript and text. Don't include opinions.\n  Then Give each key takeaways a number and keep them direct and to the point\n  VIDEO TRANSCRIPT: '''you're one of the greatest teachers of machine learning AI ever from cs231n to today what advice would you give to beginners interested in getting into machine learning beginners are often focused on like what to do and I think the focus should be more like how much you do so I I'm kind of like believer on a high level in this 10 000 hours kind of concept where you just kind of have to just pick the things where you can spend time and you you care about and you're inter

## YouTube video metadata and pass that as additional information to the LLM to improve the summary

In [4]:


video_id = re.search(r'(?<=v=)[\w-]+', url).group(0)
    print(video_id)
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        context = []
        for line in transcript:
            context.append(line['text'])
        # Print out the transcript
        #print(" ".join(context))
        content = " ".join(context)
    except Exception as e:
        raise Exception(str(e))



def print_transcript(transcript):
    for item in transcript:
        start_time = item['start']
        end_time = item['start'] + item['duration']
        text = item['text']
        print(f"{start_time} --> {end_time}: {text}\n")

if __name__ == "__main__":
    # Replace 'YOUR_VIDEO_ID' with the actual YouTube video ID
    video_id = 'I2ZK3ngNvvI'

    transcript = get_youtube_transcript(video_id)
    if transcript:
        print("Transcript:\n")
        print_transcript(transcript)
    else:
        print("No transcript found for the given video.")


Transcript:

2.76 --> 9.84: you're one of the greatest teachers of

4.92 --> 12.36: machine learning AI ever from cs231n to

9.84 --> 13.86: today what advice would you give to

12.36 --> 15.42: beginners interested in getting into

13.86 --> 19.14: machine learning

15.42 --> 20.58: beginners are often focused on like what

19.14 --> 23.279: to do and I think the focus should be

20.58 --> 24.9: more like how much you do so I I'm kind

23.279 --> 27.18: of like believer on a high level in this

24.9 --> 28.619: 10 000 hours kind of concept where

27.18 --> 30.0: you just kind of have to just pick the

28.619 --> 31.259999999999998: things where you can spend time and you

30.0 --> 32.7: you care about and you're interested in

31.26 --> 33.96: you literally have to put in 10 000

32.7 --> 35.46: hours of work

33.96 --> 37.5: um it doesn't even like matter as much

35.46 --> 39.239000000000004: like where you put it and you'll iterate

37.5 --> 40.5: and you'll improve and you'll wast